In [1]:
import numpy as np
import pandas as pd
import pysqlite3
import matplotlib.pyplot as plt
import seaborn as sns

%pylab inline

#sqlite connection
conn = pysqlite3.connect('kayzen.db')

Populating the interactive namespace from numpy and matplotlib


### Data Manipulation

In [61]:
stark_imp = pd.read_csv("stark_imp (1).csv", low_memory=False)
stark_click = pd.read_csv("stark_click (1).csv")
stark_install = pd.read_csv("stark_install (1).csv")
country_mapping = pd.read_excel("Country Mapping (2).xlsx", encoding='latin')

#Dropping Duplicates: Unique Id
stark_imp.drop_duplicates(subset=['Unique Id'], inplace=True)
stark_click.drop_duplicates(subset=['Unique Id'], inplace=True)
stark_install.drop_duplicates(subset=['Unique Id'], inplace=True)


#Adding Datetime column from 'Epoch Timestamp'
stark_click['Datetime'] = pd.to_datetime(stark_click['Epoch Timestamp'], unit='s')
stark_imp['Datetime'] = pd.to_datetime(stark_imp['Epoch Timestamp'], unit='s')
stark_install['Datetime'] = pd.to_datetime(stark_install['Epoch Timestamp'], unit='s')

# df > sql
stark_imp.to_sql('imp', conn)
stark_click.to_sql('click', conn)
stark_install.to_sql('install', conn)
country_mapping.to_sql('country',conn)

%load_ext sql
%sql sqlite:///kayzen.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Info

In [41]:
stark_install['Unique Id'].nunique()

152294

In [4]:
print(stark_imp.info(memory_usage='deep'))
print(stark_click.info(memory_usage='deep'))
print(stark_install.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276055 entries, 0 to 279920
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   REQ_CITY_ID          276055 non-null  int64         
 1   REQ_COUNTRY_ID       276055 non-null  int64         
 2   Raw User Id          276055 non-null  object        
 3   App Id               272745 non-null  object        
 4   EXCHANGE_ID          276055 non-null  int64         
 5   DEVICE_MANUFACTURER  276055 non-null  int64         
 6   DEVICE_MODEL         276055 non-null  int64         
 7   DEVICE_OS            276055 non-null  int64         
 8   OS_Version           276055 non-null  int64         
 9   USER_CARRIER         276055 non-null  int64         
 10  CAMPAIGN_ID          276055 non-null  int64         
 11  CREATIVE_ID_1        276055 non-null  int64         
 12  CREATIVE_ID_2        276055 non-null  int64         
 13  Unique Id     

#### Assumptions:
* **VTA** is enabled
* Unmatched _'REQ_COUNTRY_ID'_ from _'stark_imp'_ and _'country_code'_ from _'stark_install'_ is due to **VPN**

### Attributed installs without click - VTA

In [5]:
%%sql

WITH attr AS (
    SELECT
    [Unique Id],
    [Raw User Id],
    [Epoch Timestamp]
    FROM install
    WHERE can_claim = 1
),
    agg AS (
    SELECT
    imp.[Raw User Id],
    COUNT(imp.[Unique Id]) AS impressions,
    COUNT(c.[Unique Id]) AS clicks,
    COUNT(i.[Unique Id]) AS installs
    FROM imp 
    LEFT JOIN click c ON c.[Unique Id] = imp.[Unique Id]
    LEFT JOIN attr i ON i.[Unique Id] = imp.[Unique Id]
    GROUP BY 1
)
SELECT
[Raw User Id],
impressions,
clicks,
installs
FROM agg
WHERE clicks = 0
AND installs = 1
LIMIT 5; 

 * sqlite:///kayzen.db
Done.


Raw User Id,impressions,clicks,installs
raw_id:01174FAD-4879-4061-9DA2-9556057B99AC,6,0,1
raw_id:07201F47-FF3A-4E06-93C7-6C2E555DDDFA,1,0,1
raw_id:0768588F-6761-4356-BACE-6F8BB360E68C,1,0,1
raw_id:07F3099D-93D2-438B-ACC1-1925EE246D6E,1,0,1
raw_id:083AAB80-C71F-4777-9B7E-5F4F68BBF282,1,0,1


### Unmatched countries

In [6]:
%%sql

WITH attr AS (
    SELECT
    [Unique Id],
    [Raw User Id],
    country_code,
    [Epoch Timestamp]
    FROM install
    WHERE can_claim = 1
    ),
    countries AS (
    SELECT
    REQ_COUNTRY_ID,
    COUNTRY_NAME
    FROM country
    )
SELECT
DISTINCT imp.[Raw User Id],
cs.COUNTRY_NAME,
i.country_code
FROM imp 
LEFT JOIN countries cs ON cs.REQ_COUNTRY_ID = imp.REQ_COUNTRY_ID
LEFT JOIN click c ON c.[Unique Id] = imp.[Unique Id]
LEFT JOIN attr i ON i.[Unique Id] = imp.[Unique Id]
WHERE i.country_code <> 'US'
AND cs.COUNTRY_NAME = 'United States'
LIMIT 10; 

 * sqlite:///kayzen.db
Done.


Raw User Id,COUNTRY_NAME,country_code
raw_id:293D1D9E-E139-41CF-972D-3BADC011B1DA,United States,DE
raw_id:5AB4BBE2-8ABB-430B-9856-8037A628D88C,United States,CA
raw_id:7C97286F-55F1-4714-9D53-AA1D23957DBA,United States,None
raw_id:7F24BD95-A836-4242-A94F-C70AD71A52AC,United States,PK
raw_id:8046E433-5E9E-4EC9-A2E3-6E8F6FCBC467,United States,KZ
raw_id:93BF02B2-2A2E-47E9-AA7F-3F6123949E4C,United States,SA
raw_id:A286A5C5-C851-481D-9DF7-543FADE51484,United States,IR
raw_id:D60C360C-2391-4CD7-AB39-4925DB8832E5,United States,PR
raw_id:F772FB5D-888D-4855-8291-F8BB9556ED30,United States,DE
raw_id:FDBE2026-6FDA-4083-91E3-C71A27F52AE5,United States,KZ


### Shape

In [7]:
stark_imp.shape

(276055, 24)

In [8]:
stark_click.shape

(6541, 24)

In [9]:
stark_install.shape

(152295, 8)

### Column Names

In [10]:
stark_imp.columns

Index(['REQ_CITY_ID', 'REQ_COUNTRY_ID', 'Raw User Id', 'App Id', 'EXCHANGE_ID',
       'DEVICE_MANUFACTURER', 'DEVICE_MODEL', 'DEVICE_OS', 'OS_Version',
       'USER_CARRIER', 'CAMPAIGN_ID', 'CREATIVE_ID_1', 'CREATIVE_ID_2',
       'Unique Id', 'Epoch Timestamp', 'CREATIVE_TYPE', 'IAB_CATEGORY_ID1',
       'IAB_CATEGORY_ID2', 'IAB_CATEGORY_ID3', 'IS_WIFI', 'CAMPAIGN_TYPE',
       'CONNECTION_TYPE', 'IMP_ORIENTATION', 'Datetime'],
      dtype='object')

In [11]:
stark_click.columns

Index(['REQ_CITY_ID', 'REQ_COUNTRY_ID', 'Raw User Id', 'App Id', 'EXCHANGE_ID',
       'DEVICE_MANUFACTURER', 'DEVICE_MODEL', 'DEVICE_OS', 'OS_Version',
       'USER_CARRIER', 'CAMPAIGN_ID', 'CREATIVE_ID_1', 'CREATIVE_ID_2',
       'Unique Id', 'Epoch Timestamp', 'CREATIVE_TYPE', 'IAB_CATEGORY_ID1',
       'IAB_CATEGORY_ID2', 'IAB_CATEGORY_ID3', 'IS_WIFI', 'CAMPAIGN_TYPE',
       'CONNECTION_TYPE', 'IMP_ORIENTATION', 'Datetime'],
      dtype='object')

In [12]:
stark_install.columns

Index(['Unique Id', 'is_valid_unique_id', 'can_claim', 'country_code',
       'Raw User Id', 'DEVICE_OS', 'Epoch Timestamp', 'Datetime'],
      dtype='object')

### Head

In [13]:
stark_imp.head()

,REQ_CITY_ID,REQ_COUNTRY_ID,Raw User Id,App Id,EXCHANGE_ID,DEVICE_MANUFACTURER,DEVICE_MODEL,DEVICE_OS,OS_Version,USER_CARRIER,...,Epoch Timestamp,CREATIVE_TYPE,IAB_CATEGORY_ID1,IAB_CATEGORY_ID2,IAB_CATEGORY_ID3,IS_WIFI,CAMPAIGN_TYPE,CONNECTION_TYPE,IMP_ORIENTATION,Datetime
0,5795440,246,raw_id:74A0C6FC-F8C6-450E-87AF-188BB71337E0,1436008834,23,1317,896,12,71,0,...,1544763675,3,11.0,157.0,187.0,1,3,1,portrait,2018-12-14 05:01:15
1,5318313,246,raw_id:A11F8505-010B-4479-9D15-8A56010E272E,1436008834,23,1317,5090,12,0,0,...,1544766534,3,11.0,157.0,187.0,1,3,1,portrait,2018-12-14 05:48:54
2,5359777,246,raw_id:496F124B-96AB-485B-9B2E-2C8910FFD510,1303389751,10044,2,0,12,0,0,...,1544764471,3,NaN,NaN,NaN,1,3,1,portrait,2018-12-14 05:14:31
3,5359777,246,raw_id:496F124B-96AB-485B-9B2E-2C8910FFD510,1303389751,10044,2,0,12,0,0,...,1544764503,3,NaN,NaN,NaN,1,3,1,portrait,2018-12-14 05:15:03
4,0,93,raw_id:CA0BA044-FB83-4288-AB8D-1B15BF66ABE8,962892510,10049,1317,0,12,79,0,...,1544764911,3,325.0,NaN,NaN,1,3,1,portrait,2018-12-14 05:21:51


In [14]:
stark_click.head()

,REQ_CITY_ID,REQ_COUNTRY_ID,Raw User Id,App Id,EXCHANGE_ID,DEVICE_MANUFACTURER,DEVICE_MODEL,DEVICE_OS,OS_Version,USER_CARRIER,...,Epoch Timestamp,CREATIVE_TYPE,IAB_CATEGORY_ID1,IAB_CATEGORY_ID2,IAB_CATEGORY_ID3,IS_WIFI,CAMPAIGN_TYPE,CONNECTION_TYPE,IMP_ORIENTATION,Datetime
0,0,246,6F556808-A569-4568-B4E1-DA47CFC77784,1419954157,10044,2,0,12,0,1932,...,1544918344,3,NaN,NaN,NaN,0,3,2,portrait,2018-12-15 23:59:04
1,0,246,0A28682E-C502-439F-80D7-8E08B1372D81,962892510,10049,1317,5692,12,83,1951,...,1544918311,3,325.0,NaN,NaN,0,3,2,portrait,2018-12-15 23:58:31
2,5368361,246,EEFDF90E-6F2D-409B-BD25-D781632002C0,962892510,10049,1317,5734,12,83,0,...,1544918396,3,325.0,NaN,NaN,1,3,1,portrait,2018-12-15 23:59:56
3,4167147,246,3787D06D-BBB6-4FB1-8EED-A9A53F7C74D3,1342112505,10044,2,0,12,0,0,...,1544918074,3,NaN,NaN,NaN,1,2,1,portrait,2018-12-15 23:54:34
11,0,246,DDC19F64-CA92-4EEC-9AFC-610238C2B516,1342112505,10044,2,0,12,0,0,...,1544918275,3,NaN,NaN,NaN,1,2,1,portrait,2018-12-15 23:57:55


In [15]:
stark_install.head()

,Unique Id,is_valid_unique_id,can_claim,country_code,Raw User Id,DEVICE_OS,Epoch Timestamp,Datetime
0,NaN,0,0,US,babe71dc-c7a9-4689-bb77-f025a21851f1,android,1544831396,2018-12-14 23:49:56
1,BSylpKk,0,0,GT,d2dad60c-2542-4e02-bcb5-36fd1be20524,android,1544831407,2018-12-14 23:50:07
4,44e563b5ab59da90dfa0ba05b09c8e12,0,0,US,53F94CA3-60BA-4EA7-A70E-0C503E11ED48,ios,1544831404,2018-12-14 23:50:04
10,RlBVpKk,0,0,UK,64250355-ca7d-4254-a3af-ff245f191fa8,android,1544831409,2018-12-14 23:50:09
12,grRVpKk,0,0,PL,f981ddc4-3717-44aa-85f6-3bbd0cdb6b69,android,1544831411,2018-12-14 23:50:11


In [16]:
country_mapping.head()

,REQ_COUNTRY_ID,COUNTRY_NAME
0,1,Null
1,2,Unknown
2,11,Afghanistan
3,12,Ã…land Islands
4,13,Albania


#### Describe

In [17]:
stark_imp.describe()

,REQ_CITY_ID,REQ_COUNTRY_ID,EXCHANGE_ID,DEVICE_MANUFACTURER,DEVICE_MODEL,DEVICE_OS,OS_Version,USER_CARRIER,CAMPAIGN_ID,CREATIVE_ID_1,CREATIVE_ID_2,Epoch Timestamp,CREATIVE_TYPE,IAB_CATEGORY_ID1,IAB_CATEGORY_ID2,IAB_CATEGORY_ID3,IS_WIFI,CAMPAIGN_TYPE,CONNECTION_TYPE
count,2.760550e+05,276055.000000,276055.000000,276055.000000,276055.000000,276055.0,276055.000000,276055.000000,276055.000000,2.760550e+05,2.760550e+05,2.760550e+05,276055.000000,105784.000000,97735.000000,25250.000000,276055.000000,276055.000000,276055.000000
mean,3.455954e+06,243.386314,7199.379214,688.193204,1800.437637,12.0,29.276362,535.686664,134232.134064,5.025245e+07,5.027745e+07,1.544945e+09,2.997022,148.120869,242.736000,203.062257,0.695843,2.415921,1.277032
std,2.260764e+06,21.262341,4518.344503,656.874808,2440.108930,0.0,38.707237,868.420803,553.811608,2.637450e+03,2.867444e+03,1.012706e+05,0.077113,88.114415,104.616211,43.972478,0.460050,0.719834,0.476876
min,0.000000e+00,0.000000,11.000000,2.000000,0.000000,12.0,0.000000,0.000000,132889.000000,5.024354e+07,5.026228e+07,1.544746e+09,1.000000,11.000000,12.000000,43.000000,0.000000,1.000000,0.000000
25%,0.000000e+00,246.000000,23.000000,2.000000,0.000000,12.0,0.000000,0.000000,134047.000000,5.025335e+07,5.027778e+07,1.544871e+09,3.000000,68.000000,187.000000,187.000000,0.000000,2.000000,1.000000
50%,4.574324e+06,246.000000,10044.000000,1317.000000,0.000000,12.0,0.000000,0.000000,134255.000000,5.025335e+07,5.027785e+07,1.544930e+09,3.000000,157.000000,187.000000,187.000000,1.000000,3.000000,1.000000
75%,5.105127e+06,246.000000,10044.000000,1317.000000,5090.000000,12.0,81.000000,1932.000000,134795.000000,5.025336e+07,5.027881e+07,1.545046e+09,3.000000,222.000000,387.000000,187.000000,1.000000,3.000000,2.000000
max,1.086099e+07,254.000000,10049.000000,1317.000000,5742.000000,12.0,83.000000,2149.000000,135098.000000,5.025434e+07,5.028153e+07,1.545153e+09,3.000000,387.000000,387.000000,387.000000,1.000000,3.000000,2.000000


In [18]:
stark_click.describe()

,REQ_CITY_ID,REQ_COUNTRY_ID,EXCHANGE_ID,DEVICE_MANUFACTURER,DEVICE_MODEL,DEVICE_OS,OS_Version,USER_CARRIER,CAMPAIGN_ID,CREATIVE_ID_1,CREATIVE_ID_2,Epoch Timestamp,CREATIVE_TYPE,IAB_CATEGORY_ID1,IAB_CATEGORY_ID2,IAB_CATEGORY_ID3,IS_WIFI,CAMPAIGN_TYPE,CONNECTION_TYPE
count,6.541000e+03,6541.000000,6541.000000,6541.000000,6541.000000,6541.0,6541.000000,6541.000000,6541.000000,6.541000e+03,6.541000e+03,6.541000e+03,6541.000000,4401.000000,4264.000000,1270.000000,6541.000000,6541.000000,6541.000000
mean,3.618515e+06,244.542272,3810.335576,1026.095704,3226.608164,12.0,31.105794,419.653264,134396.166030,5.025303e+07,5.027795e+07,1.544938e+09,2.987769,165.808907,272.690197,220.137795,0.671151,2.719768,1.319676
std,2.149070e+06,15.891545,4859.471497,545.856568,2448.327199,0.0,39.480338,799.524214,514.141438,1.614409e+03,2.320451e+03,1.009642e+05,0.155933,79.075574,108.025867,38.745567,0.469831,0.552633,0.476121
min,0.000000e+00,0.000000,11.000000,2.000000,0.000000,12.0,0.000000,0.000000,132889.000000,5.024354e+07,5.026228e+07,1.544746e+09,1.000000,11.000000,12.000000,157.000000,0.000000,1.000000,0.000000
25%,0.000000e+00,246.000000,23.000000,1317.000000,0.000000,12.0,0.000000,0.000000,134047.000000,5.025335e+07,5.027785e+07,1.544851e+09,3.000000,157.000000,187.000000,187.000000,0.000000,3.000000,1.000000
50%,4.744870e+06,246.000000,23.000000,1317.000000,5090.000000,12.0,0.000000,0.000000,134795.000000,5.025335e+07,5.027881e+07,1.544926e+09,3.000000,187.000000,187.000000,187.000000,1.000000,3.000000,1.000000
75%,4.931972e+06,246.000000,10044.000000,1317.000000,5090.000000,12.0,82.000000,0.000000,134795.000000,5.025336e+07,5.027881e+07,1.545016e+09,3.000000,222.000000,387.000000,250.000000,1.000000,3.000000,2.000000
max,8.299576e+06,246.000000,10049.000000,1317.000000,5742.000000,12.0,83.000000,1951.000000,135098.000000,5.025434e+07,5.028153e+07,1.545143e+09,3.000000,387.000000,387.000000,387.000000,1.000000,3.000000,2.000000


In [19]:
stark_install.describe()

,is_valid_unique_id,can_claim,Epoch Timestamp
count,152295.000000,152295.000000,1.522950e+05
mean,0.008017,0.008017,1.544965e+09
std,0.089180,0.089180,1.169962e+05
min,0.000000,0.000000,1.544746e+09
25%,0.000000,0.000000,1.544875e+09
50%,0.000000,0.000000,1.544962e+09
75%,0.000000,0.000000,1.545063e+09
max,1.000000,1.000000,1.545178e+09


#### _'CREATIVE_ID_2'_ is a **variant** of _'CREATIVE_ID_1'_
creative type > creative id 1 > creative id 2

In [48]:
%%sql

SELECT
CREATIVE_ID_1,
CREATIVE_ID_2,
COUNT([Unique Id])
FROM imp 
GROUP BY 1,2
ORDER BY 1
LIMIT 10; 

 * sqlite:///kayzen.db
Done.


CREATIVE_ID_1,CREATIVE_ID_2,COUNT([Unique Id])
50243545,50280862,8174
50244446,50262280,24
50244447,50262281,387
50245500,50280861,11232
50245525,50263556,34
50245525,50263629,810
50245526,50263557,31
50245526,50263630,309
50245527,50263559,28
50245527,50263631,408


#### is_wifi and Connection type 

In [50]:
%%sql

WITH attr AS (
    SELECT
    [Unique Id],
    [Raw User Id],
    [Epoch Timestamp]
    FROM install
    WHERE can_claim = 1
    )
SELECT
imp.CONNECTION_TYPE,
imp.IS_WIFI,
COUNT(imp.[Unique Id]) AS impressions
FROM imp 
LEFT JOIN click c ON c.[Unique Id] = imp.[Unique Id]
LEFT JOIN attr i ON i.[Unique Id] = imp.[Unique Id]
GROUP BY 1,2
ORDER BY 1;

 * sqlite:///kayzen.db
Done.


CONNECTION_TYPE,IS_WIFI,impressions
0,0,3744
1,1,192091
2,0,80220


#### IAB categories - most values are null

In [52]:
pd.unique(stark_imp[['IAB_CATEGORY_ID1', 'IAB_CATEGORY_ID2', 'IAB_CATEGORY_ID2']].values.ravel())

array([ 11., 157.,  nan, 325., 187., 387.,  17., 222., 331.,  84.,  43.,
        92., 250., 162.,  12.,  68., 241., 339.,  19., 302., 251., 205.,
       182., 371., 164., 180., 295., 231., 138., 367.,  76., 209.,  16.,
        56., 223., 158., 123., 136., 296.])

In [53]:
stark_imp['IAB_CATEGORY_ID1'].isna().sum()

170271

In [57]:
stark_imp['IAB_CATEGORY_ID1'].notnull().sum()

105784

In [54]:
stark_imp['IAB_CATEGORY_ID2'].isna().sum()

178320

In [59]:
stark_imp['IAB_CATEGORY_ID2'].notnull().sum()

97735

In [55]:
stark_imp['IAB_CATEGORY_ID3'].isna().sum()

250805

In [60]:
stark_imp['IAB_CATEGORY_ID3'].notnull().sum()

25250

#### Unique Values per Column

In [20]:
stark_imp['REQ_COUNTRY_ID'].unique()

array([246,  93,   0, 116,  50, 122,  42, 189, 100, 193, 185, 167,  11,
       178,  96, 245, 124, 133,  70,  71, 169,  24, 208,  86, 154, 238,
       130, 244, 219, 114, 190,  91, 243,  56, 140, 210, 249,  76, 129,
       231, 205, 216, 254, 181, 145,  45,  28, 211, 192,  72, 113, 177,
        92, 172, 187,  68,  14, 242, 117,  59, 228, 253,  26, 120, 125,
        94, 119, 166, 149, 110])

In [21]:
stark_imp['Raw User Id'].nunique()

121970

In [22]:
stark_imp['App Id'].nunique()

2569

In [23]:
stark_imp['EXCHANGE_ID'].unique()

array([   23, 10044, 10049, 10038, 10031, 10046, 10048, 10036,    18,
          12,    11,    34, 10047, 10032,    14,    20, 10045])

In [24]:
stark_imp['DEVICE_MANUFACTURER'].unique()

array([1317,    2])

In [25]:
stark_imp['DEVICE_MODEL'].unique()

array([ 896, 5090,    0, 5726, 5735, 5691, 5689, 5734, 5692, 5741, 5720,
       5690, 5694, 5742, 5693, 5037, 1046,  977])

In [26]:
stark_imp['DEVICE_OS'].unique()

array([12])

In [27]:
stark_imp['USER_CARRIER'].unique()

array([   0, 1947, 1932, 1951, 1949, 1946, 1931, 1939, 1935, 1938, 1934,
       1940, 2149, 2142])

In [28]:
stark_imp['CAMPAIGN_ID'].unique()

array([134686, 134687, 134851, 133314, 133534, 133535, 134899, 134795,
       133586, 133796, 133830, 133833, 133853, 134047, 134105, 134242,
       134255, 134266, 134288, 135018, 133387, 133500, 132889, 135093,
       135094, 135097, 135095, 135096, 135098])

In [29]:
stark_imp['CREATIVE_ID_1'].unique()

array([50250928, 50250936, 50250940, 50250929, 50253356, 50254305,
       50245527, 50245529, 50245525, 50254304, 50253358, 50254303,
       50253355, 50253354, 50253353, 50254333, 50244447, 50245528,
       50253090, 50245526, 50247954, 50247956, 50247953, 50247951,
       50254334, 50254338, 50253086, 50250933, 50247952, 50254335,
       50254337, 50254336, 50247955, 50247957, 50244446, 50245500,
       50243545, 50250935, 50250932, 50250934, 50253349, 50250937,
       50250930, 50254332, 50254339, 50247958, 50250931, 50250939,
       50250938, 50253348])

In [30]:
stark_imp['CREATIVE_ID_2'].nunique()

215

In [31]:
stark_imp['Unique Id'].nunique()

276055

In [32]:
stark_imp['CREATIVE_TYPE'].unique()

array([3, 1])

In [33]:
stark_imp['IAB_CATEGORY_ID1'].unique()

array([ 11.,  nan, 325., 187., 387., 157., 222.,  84.,  43.,  17.,  92.,
       162., 331.,  12., 241.,  19.,  68., 182., 205., 302., 180., 250.,
       164., 295., 231.,  76., 138., 209., 371., 339.,  56., 223., 123.,
       296.])

In [34]:
stark_imp['IAB_CATEGORY_ID2'].unique()

array([157.,  nan, 187.,  17., 387., 331.,  43., 250.,  68., 325., 339.,
       222.,  12., 302., 162., 251., 205., 371., 164., 180., 295., 231.,
       241., 138., 367.,  84.,  92., 182., 209.,  19.,  16., 158., 136.,
       296., 223.])

In [35]:
stark_imp['IAB_CATEGORY_ID3'].unique()

array([187.,  nan, 331., 180.,  43., 250., 251., 157., 209., 222., 295.,
       241., 189., 302., 371., 376., 231., 205., 138., 387., 245., 137.,
        84., 325.])

In [36]:
stark_imp['IS_WIFI'].unique()

array([1, 0])

In [37]:
stark_imp['CAMPAIGN_TYPE'].unique()

array([3, 2, 1])

In [38]:
stark_imp['CONNECTION_TYPE'].unique()

array([1, 2, 0])

In [39]:
stark_imp['IMP_ORIENTATION'].unique()

array(['portrait', 'landscape'], dtype=object)